# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [55]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#abrir navegador
navegador = webdriver.Chrome()


In [56]:
#pesquisar cotação dolar
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('contação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

In [57]:
#copiar cotação dolar
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

5.338299999999999


In [58]:
#pesquisar cotação euro
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('contação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

In [59]:
#pegar cotação euro
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

6.471888005


In [60]:
#entrar no site melhor ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20306%2C10.')


In [61]:
#pegar cotação ouro
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

304.29


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [62]:
import pandas as pd

tabela_produtos = pd.read_excel('Produtos.xlsx')
display(tabela_produtos)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [64]:
#1 atualizar cotações
#atualizar varios valores de uma vez
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

#atualizar valores da tabela
tabela_produtos['Preço Base Reais'] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map('{:.2f}'.format)
display(tabela_produtos)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.338300,5338.246617,1.40,7473.55
1,Carro Renault,4500.00,Euro,6.471888,29123.496022,2.00,58246.99
2,Notebook Dell,899.99,Dólar,5.338300,4804.416617,1.70,8167.51
3,IPhone,799.00,Dólar,5.338300,4265.301700,1.70,7251.01
4,Carro Fiat,3000.00,Euro,6.471888,19415.664015,1.90,36889.76
5,Celular Xiaomi,480.48,Dólar,5.338300,2564.946384,2.00,5129.89
6,Joia 20g,20.00,Ouro,304.290000,6085.800000,1.15,6998.67


### Agora vamos exportar a nova base de preços atualizada

In [65]:
tabela_produtos.to_excel('Produtos_Atualizados.xlsx', index=False)

In [67]:
atuais = pd.read_excel('Produtos_Atualizados.xlsx')
display(atuais)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.338300,5338.246617,1.40,7473.55
1,Carro Renault,4500.00,Euro,6.471888,29123.496022,2.00,58246.99
2,Notebook Dell,899.99,Dólar,5.338300,4804.416617,1.70,8167.51
3,IPhone,799.00,Dólar,5.338300,4265.301700,1.70,7251.01
4,Carro Fiat,3000.00,Euro,6.471888,19415.664015,1.90,36889.76
5,Celular Xiaomi,480.48,Dólar,5.338300,2564.946384,2.00,5129.89
6,Joia 20g,20.00,Ouro,304.290000,6085.800000,1.15,6998.67
